In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), '../'))

from evaluation.evaluation_harness import WarmFineTuneEvaluator
from pprint import pprint

from dataset.data_loader import DataLoader, playtime_forever_users_games_edge_scoring_function, LinearNormalizer, GaussianNormalizer, PercentileNormalizer, never_remove_edge, remove_zero_playtime_edge
from models.random_model import RandomModel
from models.common_neighbors_model import CommonNeighbors
from models.popularity_model import GamePopularityModel
from models.ncf_model import NCFModel

In [2]:
data_loader = DataLoader(cache_local_dataset=True, users_games_edge_scoring_function = playtime_forever_users_games_edge_scoring_function, user_game_edge_embeddings = ['playtime_forever'], score_normalizers = [GaussianNormalizer(1.0, 1.0)])
data_loader.load_random_edge_train_test_split(train_percentage=0.8, test_percentage=0.2, seed=0)

evaluator = WarmFineTuneEvaluator(data_loader, top_N_games_to_eval = 500, num_users_to_eval=10000, seed=0, debug=True)

In [3]:
random_model_initializer = lambda: RandomModel()
evaluator.reset(random_model_initializer, network_save_file = 'evaluation_warm_fine_tune_test_random_model')
evaluator.plot_top_N_recall_percentiles(10)
evaluator.plot_log_user_percentile_positional_error()
evaluator.compute_mean_positional_error()
evaluator.compute_top_N_recall(10)
evaluator.plot_roc_curve()
evaluator.compute_auc_roc()
evaluator.plot_user_rank_roc_curve()
evaluator.compute_user_rank_auc_roc()
evaluator.plot_top_N_hit_percentage_percentiles(10)
evaluator.compute_top_N_hit_percentage(10)
evaluator.save_metrics('warm_fine_tune_test_random_model', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Preparing model.


Fine tuning: 100%|██████████| 10000/10000 [00:05<00:00, 1725.04it/s]


Done preparing model.
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [03:08<00:00, 53.15it/s]


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
random
{'auc_roc': 0.43791067106954584,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 21.014732105055195,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.00186,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall': 0.0003125908572216051,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'user_rank_auc_roc': 0.49873420488475684,
 'user_rank_roc_figure': <Figure size 640x480 with 1 Axes>}



In [4]:
popularity_model_initializer = lambda: GamePopularityModel()
evaluator.reset(popularity_model_initializer, network_save_file = 'evaluation_warm_fine_tune_test_popularity_model')
evaluator.plot_top_N_recall_percentiles(10)
evaluator.plot_log_user_percentile_positional_error()
evaluator.compute_mean_positional_error()
evaluator.compute_top_N_recall(10)
evaluator.plot_roc_curve()
evaluator.compute_auc_roc()
evaluator.plot_user_rank_roc_curve()
evaluator.compute_user_rank_auc_roc()
evaluator.plot_top_N_hit_percentage_percentiles(10)
evaluator.compute_top_N_hit_percentage(10)
evaluator.save_metrics('warm_fine_tune_test_popularity_model', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Preparing model.


Fine tuning: 100%|██████████| 10000/10000 [00:05<00:00, 1955.96it/s]


Done preparing model.
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [01:28<00:00, 112.60it/s]


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
game_popularity
{'auc_roc': 0.5515575916825022,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 18.282347335138635,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.18882,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall': 0.03173301379601262,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'user_rank_auc_roc': 0.6601328958203395,
 'user_rank_roc_figure': <Figure size 640x480 with 1 Axes>}



In [5]:
common_neighbors_initializer = lambda: CommonNeighbors()
evaluator.reset(common_neighbors_initializer, network_save_file = 'evaluation_warm_fine_tune_test_common_neighbors')
evaluator.plot_top_N_recall_percentiles(10)
evaluator.plot_log_user_percentile_positional_error()
evaluator.compute_mean_positional_error()
evaluator.compute_top_N_recall(10)
evaluator.plot_roc_curve()
evaluator.compute_auc_roc()
evaluator.plot_user_rank_roc_curve()
evaluator.compute_user_rank_auc_roc()
evaluator.plot_top_N_hit_percentage_percentiles(10)
evaluator.compute_top_N_hit_percentage(10)
evaluator.save_metrics('warm_fine_tune_test_common_neighbors', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Preparing model.


Fine tuning: 100%|██████████| 10000/10000 [00:05<00:00, 1915.63it/s]


Done preparing model.
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [12:00<00:00, 13.88it/s]


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
common_neighbors
{'auc_roc': 0.8613416577261556,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 17.23173562413738,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.17637202769138155,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall': 0.029543197199454814,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'user_rank_auc_roc': 0.6511261844784793,
 'user_rank_roc_figure': <Figure size 640x480 with 1 Axes>}



In [6]:
cf_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 100, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-3, mlp_hidden_layer_sizes = [], seed=0, model_type='cf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5)
evaluator.reset(cf_initializer, network_save_file = 'evaluation_warm_fine_tune_test_cf')
evaluator.plot_top_N_recall_percentiles(10)
evaluator.plot_log_user_percentile_positional_error()
evaluator.compute_mean_positional_error()
evaluator.compute_top_N_recall(10)
evaluator.plot_roc_curve()
evaluator.compute_auc_roc()
evaluator.plot_user_rank_roc_curve()
evaluator.compute_user_rank_auc_roc()
evaluator.plot_top_N_hit_percentage_percentiles(10)
evaluator.compute_top_N_hit_percentage(10)
evaluator.save_metrics('warm_fine_tune_test_cf', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Preparing model.
Known Game Embeddings:  []
Known User Embeddings:  []
Total Learnable Parameters: 11340200


Fine tuning: 100%|██████████| 10000/10000 [00:05<00:00, 1958.19it/s]


Done preparing model.
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [10:22<00:00, 16.05it/s]


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
neural_collborative_filtering_cf
{'auc_roc': 0.46228022941499874,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 21.01569287080213,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.00218,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall': 0.0003663699294317737,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'user_rank_auc_roc': 0.5022202157374727,
 'user_rank_roc_figure': <Figure size 640x480 with 1 Axes>}



In [7]:
gcf_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 100, batch_percent = 0.1, learning_rate = 1e-3, mlp_hidden_layer_sizes = [], seed=0, model_type='gcf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5)
evaluator.reset(gcf_initializer, network_save_file = 'evaluation_warm_fine_tune_test_gcf')
evaluator.plot_top_N_recall_percentiles(10)
evaluator.plot_log_user_percentile_positional_error()
evaluator.compute_mean_positional_error()
evaluator.compute_top_N_recall(10)
evaluator.plot_roc_curve()
evaluator.compute_auc_roc()
evaluator.plot_user_rank_roc_curve()
evaluator.compute_user_rank_auc_roc()
evaluator.plot_top_N_hit_percentage_percentiles(10)
evaluator.compute_top_N_hit_percentage(10)
evaluator.save_metrics('warm_fine_tune_test_gcf', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Preparing model.
Known Game Embeddings:  []
Known User Embeddings:  []
Total Learnable Parameters: 11340301


Fine tuning: 100%|██████████| 10000/10000 [00:05<00:00, 1954.48it/s]


Done preparing model.
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [10:20<00:00, 16.11it/s]


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
neural_collborative_filtering_gcf
{'auc_roc': 0.5625197459538817,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 20.355682030626348,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.006609933900660993,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall': 0.0011108739603412955,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'user_rank_auc_roc': 0.5347960893801665,
 'user_rank_roc_figure': <Figure size 640x480 with 1 Axes>}



In [8]:
mlp_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 100, batch_percent = 0.1, learning_rate = 1e-3, weight_decay=1e-3, mlp_hidden_layer_sizes = [128, 128, 128], seed=0, model_type='mlp', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5)
evaluator.reset(mlp_initializer, network_save_file = 'evaluation_warm_fine_tune_test_mlp')
evaluator.plot_top_N_recall_percentiles(10)
evaluator.plot_log_user_percentile_positional_error()
evaluator.compute_mean_positional_error()
evaluator.compute_top_N_recall(10)
evaluator.plot_roc_curve()
evaluator.compute_auc_roc()
evaluator.plot_user_rank_roc_curve()
evaluator.compute_user_rank_auc_roc()
evaluator.plot_top_N_hit_percentage_percentiles(10)
evaluator.compute_top_N_hit_percentage(10)
evaluator.save_metrics('warm_fine_tune_test_mlp', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Preparing model.
Known Game Embeddings:  []
Known User Embeddings:  []
Total Learnable Parameters: 11399081


Training:  25%|██▌       | 10/40 [09:39<29:30, 59.03s/it]

In [ ]:
ncf_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 100, batch_percent = 0.1, learning_rate = 1e-3, weight_decay=1e-3, mlp_hidden_layer_sizes = [128, 128, 128], seed=0, model_type='ncf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5)
evaluator.reset(ncf_initializer, network_save_file = 'evaluation_warm_fine_tune_test_ncf')
evaluator.plot_top_N_recall_percentiles(10)
evaluator.plot_log_user_percentile_positional_error()
evaluator.compute_mean_positional_error()
evaluator.compute_top_N_recall(10)
evaluator.plot_roc_curve()
evaluator.compute_auc_roc()
evaluator.plot_user_rank_roc_curve()
evaluator.compute_user_rank_auc_roc()
evaluator.plot_top_N_hit_percentage_percentiles(10)
evaluator.compute_top_N_hit_percentage(10)
evaluator.save_metrics('warm_fine_tune_test_ncf', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()